In [1]:
import pdfplumber
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import pickle
import numpy as np
import re
from scipy import spatial

In [2]:
import dtale
#handy pandas visualizer, when on local Jupyter Server just run dtale.show(df)
#from Docker container visual is available at http://localhost:40000/

In [3]:
path = '../datasets/resume_samples.txt'
df = pd.DataFrame()
counter = 0
id_list = []
title_list = []
skill_list = []
#exp_list = []
with open(path, 'r',encoding="utf8", errors='backslashreplace') as f:
    for line in f.readlines():
        
        splitted = line.split(':::')
        title = splitted[1].split(';')[0]
        if 'data scientist' in title.lower():
            splitted = splitted[2].lower()
            splitted = splitted.replace('\\xa0',';;;')
            splitted = splitted.replace('*',';;;')
            splitted = splitted.replace('\\x95',';;;')
            splitted = splitted.replace('education',';;;')
            #splitted = splitted.replace(',',';;;')
            splitted = splitted.split(';;;')
            
            for skill in splitted[1:]:
                if (len(skill)>1) & ('responsibilities:' not in skill.lower()):
                    id_list.append(counter)
                    
                    title_list.append(title)
                    skill_list.append(skill)
            counter+=1
df['id']=id_list
df['title']=title_list
df['skill']=skill_list

In [4]:
df

,id,title,skill
0,0,Data Scientist/Analytics Consultant,"design sql, r & python scripts to directly qu..."
1,0,Data Scientist/Analytics Consultant,aid in the development of predictive modeling...
2,0,Data Scientist/Analytics Consultant,test model fit using cross-validation and boo...
3,0,Data Scientist/Analytics Consultant,develop in an scrum environment with team mem...
4,0,Data Scientist/Analytics Consultant,"model data using pandas/r dataframes, powerpi..."
...,...,...,...
5649,66,Data Scientist,"development methodologies agile/scrum, uml, d..."
5650,66,Data Scientist,"build tools jenkins, toad, sql loader, maven,..."
5651,66,Data Scientist,reporting tools ms office (word/excel/power p...
5652,66,Data Scientist,"databases microsoft sql server 2008,2010/2012..."


In [5]:
def preprocess_for_bert(string):
    string = string.strip("\n.’:")
    string = string.strip("’")
    string = string.strip("\\n")
    string = string.replace("/"," ")
    return(string)

In [6]:
k21 = pickle.load(open('../Experiments/k21.sav', 'rb'))
objectives = []
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
skill_list = []
for s in df['skill'].to_list():
    skill_list.append(preprocess_for_bert(s))
embeddings = model.encode(skill_list)
clusters =  k21.predict(np.array(embeddings.tolist()))
df['cluster']=clusters

In [8]:
print(clusters[1])

16


In [9]:
results = []
for index, row in df.iterrows():
    #print(clusters[index])
    #print(embeddings[index])
    #k21.cluster_centers_[1]
    result = 1 - spatial.distance.cosine(k21.cluster_centers_[clusters[index]], embeddings[index])
    results.append(result)
    #print(df[skill].loc[index])
df['similarity_to_cluster']=results

In [10]:
k20_bigrams = pickle.load(open('../Experiments/k20_bigrams', 'rb'))

In [11]:
bigrams = []
for cluster in clusters:
    bigrams.append(k20_bigrams[cluster])
df['bigrams_k20']=bigrams

In [12]:
d=dtale.show(df)

2021-09-20 20:34:36,442 - INFO     - NumExpr defaulting to 2 threads.


In [13]:
dtale.instances()

To gain access to an instance object simply pass the value from 'ID' to dtale.get_instance(ID)

ID Name                                    URL
 1      http://f468fa972f9e:40000/dtale/main/1
        http://f468fa972f9e:40000/dtale/main/1


In [14]:
#df['cluster'].value_counts()
for c, count in df['cluster'].value_counts().iteritems():
    print(str(c)+" "+str(count)+" "+k20_bigrams[c])
    print('-----------------------')
    
for i in range(0,20):
    if i not in df['cluster'].to_list():
         print('missing '+str(i))

15 1427 data visualization, large datasets, experience data, knowledge data, data visualisation, large data, big data, understanding data, data engineering, data pipelines, 
-----------------------
3 1061 software development, software engineering, best practices, data science, experience working, processing frameworks, frameworks spark, tools (e.g., standard software, ci cd,, 
-----------------------
11 847 develop processes, processes tools, tools monitor, model performance, performance data, data accuracy, analyze data, data analysis, ability develop, develop experimental, 
-----------------------
16 492 machine learning, learning techniques, learning algorithms, understanding machine, experience machine, deep learning, learning experience, strong knowledge, knowledge machine, machine learning,, 
-----------------------
7 383 relational databases, experience sql, knowledge sql, sql queries, experience databases, advanced sql, sql skills, experience relational, sql experience, python

In [15]:
cluster_matrix = []

for id in np.unique(df[['id']].values):
    cluster_counter = []
    filtered = df[df['id']==id]
    for cluster in range(0,20):
        c=0
        for index, row in filtered.iterrows():
            if row['cluster']==cluster:
                c+=1/len(filtered)
        cluster_counter.append(c)
    cluster_matrix.append(cluster_counter)

In [16]:
df_resume  = pd.DataFrame()
df_resume['id']=np.unique(df[['id']].values)
df_resume['matrix']=cluster_matrix
#df_resume = df_resume.append(pd.DataFrame(df_resume['matrix'].tolist()))
foo=pd.DataFrame(df_resume['matrix'].tolist())
df_resume = pd.concat([df_resume,foo], axis=1)
counted = df_resume.iloc[:,2:].astype(bool).sum(axis=0).sort_values(ascending=False)

In [17]:
scored = df_resume.iloc[:,2:].sum(axis=0).sort_values(ascending=False)

In [18]:
for cluster, score in scored.iteritems():
    print("cluster "+str(cluster)+" scored "+str(score)+" Most common bigrams of this cluster are "+k20_bigrams[cluster])
    print('---------------------')

cluster 15 scored 14.830993416249223 Most common bigrams of this cluster are data visualization, large datasets, experience data, knowledge data, data visualisation, large data, big data, understanding data, data engineering, data pipelines, 
---------------------
cluster 3 scored 13.658230137730028 Most common bigrams of this cluster are software development, software engineering, best practices, data science, experience working, processing frameworks, frameworks spark, tools (e.g., standard software, ci cd,, 
---------------------
cluster 11 scored 9.86464014150455 Most common bigrams of this cluster are develop processes, processes tools, tools monitor, model performance, performance data, data accuracy, analyze data, data analysis, ability develop, develop experimental, 
---------------------
cluster 16 scored 5.301561245213319 Most common bigrams of this cluster are machine learning, learning techniques, learning algorithms, understanding machine, experience machine, deep learning

2021-09-20 20:40:42,921 - ERROR    - Exception occurred while processing request: min() arg is an empty sequence
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 105, in _handle_exceptions
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 2381, in load_filtered_ranges
    overall_min = min([v["min"] for v in filtered_ranges.values()])
ValueError: min() arg is an empty sequence


Exception occurred while processing request: min() arg is an empty sequence
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 105, in _handle_exceptions
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 2381, in load_filtered_ranges
    overall_min = min([v["min"] for v in filtered_ranges.values()])
ValueError: min() arg is an empty sequence


2021-09-20 20:40:45,427 - ERROR    - Exception occurred while processing request: min() arg is an empty sequence
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 105, in _handle_exceptions
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 2381, in load_filtered_ranges
    overall_min = min([v["min"] for v in filtered_ranges.values()])
ValueError: min() arg is an empty sequence


Exception occurred while processing request: min() arg is an empty sequence
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 105, in _handle_exceptions
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 2381, in load_filtered_ranges
    overall_min = min([v["min"] for v in filtered_ranges.values()])
ValueError: min() arg is an empty sequence


...

2021-09-20 20:41:29,002 - ERROR    - Exception occurred while processing request: min() arg is an empty sequence
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 105, in _handle_exceptions
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 2381, in load_filtered_ranges
    overall_min = min([v["min"] for v in filtered_ranges.values()])
ValueError: min() arg is an empty sequence


Exception occurred while processing request: min() arg is an empty sequence
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 105, in _handle_exceptions
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 2381, in load_filtered_ranges
    overall_min = min([v["min"] for v in filtered_ranges.values()])
ValueError: min() arg is an empty sequence


2021-09-20 20:42:21,990 - ERROR    - Exception occurred while processing request: min() arg is an empty sequence
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 105, in _handle_exceptions
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 2381, in load_filtered_ranges
    overall_min = min([v["min"] for v in filtered_ranges.values()])
ValueError: min() arg is an empty sequence


Exception occurred while processing request: min() arg is an empty sequence
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 105, in _handle_exceptions
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 2381, in load_filtered_ranges
    overall_min = min([v["min"] for v in filtered_ranges.values()])
ValueError: min() arg is an empty sequence


2021-09-20 20:44:00,929 - ERROR    - Exception occurred while processing request: min() arg is an empty sequence
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 105, in _handle_exceptions
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/dtale/views.py", line 2381, in load_filtered_ranges
    overall_min = min([v["min"] for v in filtered_ranges.values()])
ValueError: min() arg is an empty sequence
